In [908]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [909]:
product_type = "ambient"
num_time_periods = 365

In [910]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [911]:

if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost = 25
max_order_exceed_multiplier = 1.5

In [912]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [913]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [914]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [915]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [916]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [917]:
def getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - cum_demand > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - cum_demand) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        #max_order = ((orders[order_date] * cu_per_tu_item) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost
        max_order = ((orders[order_date]) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost 
    return max_order 

In [918]:
def silverMeal(item):
    
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0

    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        
        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue
            cum_demand += item_demand[t]
            
            if(t==order_date+life):
                orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) 
                tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item) 
            if(math.ceil(cum_demand/cu_per_tu_item) > maximum_order_quantity_tu_item):
                penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier * variable_cost
                #penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier  
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t==len(item_demand)-1):
                    
                    orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                    #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item,variable_cost, maximum_order_quantity_tu_item)
                    tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
                    tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                    tot_fixed += fixed_cost   
                    tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                 
                    run_complete = True
                    break

            else:
                orders[order_date] = math.ceil((cum_demand- item_demand[t])/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand- item_demand[t], disposal_cost)
                tot_ordering += math.ceil((cum_demand- item_demand[t])/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)

                order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
        
    
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)


46530639.89999996
Ordering 41081439.500000104
Fixing  3926277.5000000093
Clearance  866329.9999999981
Max Order  656592.8999999997


In [919]:
buf_cost = 0
vol = 0
time_indexes
volumes = {}

for time in range(len(time_indexes)):
    for item in schedule:
        if time in schedule[item].keys():
            vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * volume_per_cu[item]

    volumes[time] = vol
    if vol > warehouse_volume:

        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost
obj_val = obj_val + buf_cost

In [920]:
obj_val

48459169.97376463

In [921]:
schedule

{'00027afc': {0: 4,
  3: 8,
  11: 5,
  16: 6,
  23: 2,
  25: 6,
  31: 7,
  38: 6,
  46: 6,
  53: 3,
  58: 2,
  63: 4,
  66: 8,
  78: 12,
  93: 3,
  99: 1,
  113: 3,
  121: 4,
  127: 12,
  142: 4,
  148: 3,
  151: 2,
  155: 4,
  162: 2,
  166: 4,
  169: 23,
  193: 2,
  197: 1,
  200: 1,
  204: 4,
  211: 5,
  221: 3,
  225: 6,
  231: 14,
  234: 25,
  256: 6,
  268: 2,
  274: 2,
  277: 2,
  282: 1,
  288: 2,
  292: 1,
  296: 2,
  301: 3,
  303: 19,
  326: 1,
  330: 1,
  336: 2,
  340: 1,
  344: 2,
  351: 1,
  354: 1,
  360: 2},
 '00084396': {0: 1,
  4: 1,
  9: 1,
  16: 1,
  18: 2,
  23: 2,
  26: 1,
  30: 1,
  36: 1,
  38: 1,
  42: 1,
  49: 1,
  52: 1,
  57: 1,
  60: 1,
  64: 1,
  68: 1,
  73: 1,
  78: 1,
  81: 1,
  85: 1,
  87: 1,
  89: 1,
  95: 1,
  99: 1,
  101: 1,
  103: 1,
  106: 1,
  108: 1,
  112: 1,
  114: 1,
  119: 1,
  126: 1,
  129: 1,
  136: 1,
  141: 1,
  144: 1,
  149: 1,
  156: 1,
  158: 1,
  163: 1,
  171: 1,
  177: 1,
  182: 5,
  185: 2,
  186: 6,
  197: 1,
  199: 1,
  204

In [922]:
max_improvement = math.inf
cost = obj_val
modified_schedule = {}
day_wise_orders = {}

buffer_penalty_tot = 0
max_order_penalty_tot = 0
fixed_ordering_cost_tot = 0 
schedule_imp = schedule 
count = 0
iterator = 0
MAX_ITER = 100
MAX_REPEAT = 3
prev_max_improvement = 0

while count < MAX_REPEAT and iterator < MAX_ITER:
    for item in schedule_imp:
        buffer_penalty = 0
        max_order_penalty = 0 
        cost_improvement = 0
        modified_orders = {}
        # Check if the item has a second order date
        if len(schedule_imp[item]) > 1:
            order_days = list(schedule_imp[item].keys())
            for i in range (0,len(order_days)-1, 2):
                day1 = i
                day2 = i + 1
                if order_days[day2] - order_days[day1] <= shelf_life[item]:
                    tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
                    vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                    if vol + volumes[day1] > warehouse_volume:
                        buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
                    if tot_orders > maximum_order_quantity_tu[item]:
                        max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
                    delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                    if delta < 0:
                        fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                        buffer_penalty_tot += buffer_penalty
                        max_order_penalty_tot += max_order_penalty
                        cost += delta
                        modified_orders[order_days[day1]] = tot_orders
                        volumes[order_days[day1]] += vol
                    else:
                        modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                        modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
            modified_schedule[item] = modified_orders
        
        else:
            if len(schedule[item]) == 1 : 
                modified_schedule[item] = schedule_imp[item]
            else:
                continue


        if cost!=0 and cost < max_improvement:
            max_improvement = cost
    
    print("Buffer penalty paid", buffer_penalty_tot)
    print("max order penalty paid", max_order_penalty_tot)
    print("fixed cost reduced by", fixed_ordering_cost_tot)
    schedule_imp = modified_schedule
    obj_val = max_improvement
    iterator +=1

    print(max_improvement)

    if(max_improvement==prev_max_improvement):
        count+=1
    prev_max_improvement = max_improvement

Buffer penalty paid 6151.936124513074
max order penalty paid 402.59999999999997
fixed cost reduced by 855911.5999999956
47609812.90988647
Buffer penalty paid 6151.936124513074
max order penalty paid 402.59999999999997
fixed cost reduced by 858217.1999999966
47607507.30988654
Buffer penalty paid 6151.936124513074
max order penalty paid 402.59999999999997
fixed cost reduced by 858500.0999999963
47607224.40988652
Buffer penalty paid 6151.936124513074
max order penalty paid 402.59999999999997
fixed cost reduced by 858517.6999999962
47607206.809886515
Buffer penalty paid 6151.936124513074
max order penalty paid 402.59999999999997
fixed cost reduced by 858517.6999999962
47607206.809886515
Buffer penalty paid 6151.936124513074
max order penalty paid 402.59999999999997
fixed cost reduced by 858517.6999999962
47607206.809886515
Buffer penalty paid 6151.936124513074
max order penalty paid 402.59999999999997
fixed cost reduced by 858517.6999999962
47607206.809886515


In [923]:
modified_schedule

{'00027afc': {0: 12,
  11: 11,
  23: 8,
  31: 13,
  46: 9,
  58: 6,
  66: 20,
  93: 4,
  113: 7,
  127: 16,
  148: 5,
  155: 6,
  166: 27,
  193: 3,
  200: 5,
  211: 8,
  225: 20,
  234: 31,
  268: 4,
  277: 3,
  288: 3,
  296: 5,
  303: 20,
  330: 3,
  340: 3,
  351: 2},
 '00084396': {0: 2,
  9: 2,
  18: 4,
  26: 2,
  36: 2,
  42: 2,
  52: 2,
  60: 2,
  68: 2,
  78: 2,
  85: 2,
  89: 2,
  99: 2,
  103: 2,
  108: 2,
  114: 2,
  126: 2,
  136: 2,
  144: 2,
  156: 2,
  163: 2,
  177: 6,
  185: 8,
  197: 2,
  204: 2,
  214: 2,
  229: 3,
  247: 2,
  259: 20,
  277: 2,
  288: 2,
  294: 2,
  305: 2,
  316: 2,
  325: 2,
  332: 2},
 '0025253f': {0: 7,
  4: 13,
  16: 10,
  25: 12,
  36: 8,
  43: 15,
  53: 12,
  60: 10,
  65: 18,
  74: 22,
  91: 15,
  100: 7,
  108: 7,
  113: 13,
  122: 10,
  130: 21,
  144: 11,
  151: 9,
  157: 11,
  165: 8,
  172: 12,
  179: 15,
  190: 13,
  200: 14,
  212: 8,
  218: 14,
  226: 19,
  239: 19,
  246: 23,
  261: 17,
  274: 9,
  283: 12,
  297: 6,
  306: 5,
  311